<a href="https://colab.research.google.com/github/michalis0/DataMining_and_MachineLearning/blob/master/week5/Classification_Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Mining and Machine Learning - Week 5
# Classification

Classification is part of **supervised learning**. The objective is to correctly assign objects to different, predifined categories or labels. An easy to understand example is classifying emails as “spam” or “not spam.”

### Table of Contents
#### 0. Summary of some important concepts
#### 1. Basic Example
* 1.1 Create Data
* 1.2 Encoding of `Group`
* 1.3 Plot `x1` and `x2` according to `target`
* 1.4 Logistic Regression and Decision Boundary

#### 2. Predict Ad Click
* 2.1 Load and explore the dataset
* 2.2 Exploratory Data Analysis
* 2.3 Logistic Regression
* 2.4 Logistic Regression with standardization

#### 3. Multi Class Regression

## 0. Summary of some important concepts

Suppose you have a sample of 100 people, 20 of whom have purchased a certain product. You want to predict whether a person will `buy` (1) or `not buy` (0) the product based on her characteristics (e.g. male/female, age, etc.).

### A. Base rate
Represents the degree of accuracy you would obtain without using an algorithm (i.e. the prior probability of the most common class). In the example, p(`buy`) = 0.2 and p(`not buy`) = 0.8. This means that you would obtain an accuracy of 80% without using an algorithm if you classify each person as `not buy`. Clearly, our approach should outperform the base rate.

### B. Accuracy
This is the number of correct decisions for a model out of the total number of decisions. This should be greater than the base rate.

### C. True positive, true negative, false positive, false negative
* True positive: True label is positive (`buy`) and algorithm classifies as positive (`buy`).
* True negative: True label is negative (`not buy`) and algorithm classifies as negative (`not buy`).
* False positive: True label is negative (`not buy`), but algorithm classifies as positive (`buy`).
* False negative: True label is positive (`buy`), but algorithm classifies as negative (`not buy`).

We can summarize these concepts using the confusion matrix.



In [55]:
# Import required packages
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.utils.multiclass import unique_labels

%matplotlib inline
sns.set_style("dark")

## 1. Basic Example

### 1.1 Create Data
For this example, we create data from scratch. We have a categorical variable (`Group`), two numerical variables (`x1` and `x2`) and the `target` is a binary variable. 

In [ ]:
sample = [
    ["A", 0.1, 0.2, 0],
    ["A", 0.3, 0.05, 0],
    ["B", 0.3, 0.2, 0],
    ["D", 0.7, 0.65, 1],
    ["B", 0.25, 0.3, 0],
    ["A", 0.85, 0.55, 1],
    ["C", 0.1, 0.45, 0],
    ["C", 0.9, 0.85, 1],
    ["D", 0.95, 0.55, 1],
    ["B", 0.8, 0.8, 1]
]

df = pd.DataFrame(sample, columns=["Group", "x1", "x2", "target"])
df

### 1.2 Encoding of `Group`

We illustrate how to encode categorical features using:
* **One-hot encoding**: create a dummy variable for each category.
* **Label encoding**: assign integers to the different categories. Useful for ordered data.



In [ ]:
# One-hot encoding
one_hot = OneHotEncoder()
cat_to_onehot = one_hot.fit_transform(df[["Group"]]).toarray()
cat_to_onehot = pd.DataFrame(cat_to_onehot, columns=one_hot.categories_)
cat_to_onehot

In [ ]:
# Label encoding
le = LabelEncoder()
cat_to_label = pd.Series(le.fit_transform(df["Group"]))
cat_to_label

In [ ]:
le.classes_

### 1.3 Plot `x1` and `x2` according to `target`
We are now interested in predicting `target` based on `x1` and `x2`. We first generate a plot.

In [ ]:
df.plot.scatter("x1", "x2", c="target", colormap="coolwarm_r")

We can see a clear separation. Points with low value of `x1` and `x2` are in the first class (`target` = 0) and points with high value of `x1` and `x2` are in the second class (`target` = 1). We can further compute the base rate (i.e the probability of the most common class).

In [ ]:
# Base rate
max(len(df[df["target"] == 0]) / len(df), len(df[df["target"] == 1]) / len(df))

### 1.4 Logistic Regression and Decision Boundary
We now fit a logistic regression and generate a plot showing the decision boundary of the model.

In [ ]:
# Fit model
X = df[["x1", "x2"]].values
y = df["target"]
LR = LogisticRegressionCV()
LR.fit(X, y)

# Plot
x_min, x_max = X[:, 0].min() - .1, X[:, 0].max() + .1
y_min, y_max = X[:, 1].min() - .1, X[:, 1].max() + .1
h = .005  # step size in the mesh
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
Z = LR.predict(np.c_[xx.ravel(), yy.ravel()])

# Put the result into a color plot
Z = Z.reshape(xx.shape)
plt.figure(1, figsize=(15, 10))
plt.pcolormesh(xx, yy, Z, cmap=plt.cm.Paired)

# Plot also the training points and two new points (p1 and p2)
plt.scatter(X[:, 0], X[:, 1], c=y, edgecolors='k', cmap=plt.cm.Paired)
plt.scatter(0.4, 0.4, c='black') # p1
plt.scatter(0.5, 0.5, c='black') # p2
plt.text(0.37, 0.37, 'p1')
plt.text(0.51, 0.5, 'p2')
plt.xlabel('x1', fontsize=15)
plt.ylabel('x2', fontsize=15)

plt.xlim(xx.min(), xx.max())
plt.ylim(yy.min(), yy.max())
plt.xticks(())
plt.yticks(())

plt.show()

In [ ]:
# Prediction for p1 and p2
p = pd.DataFrame([[0.4, 0.4], 
                 [0.5, 0.5]], columns=["x1", "x2"])
LR.predict(p)

In [ ]:
# Score of the model --> better than base rate :)
LR.score(X, y)
# accuracy_score(LR.predict(X), y)

## 2. Predict Ad Click
In this section, we use **Logistic Regression** to predict whether or not a particular Internet user will click on an advertisement. You can find the data set [here](https://www.kaggle.com/fayomi/advertising).

### 2.1 Load and explore the dataset

In [ ]:
data = pd.read_csv("https://raw.githubusercontent.com/michalis0/DataMining_and_MachineLearning/master/week5/data/advertising.csv")
data

The data set has 1000 rows and 10 features:

* `Daily Time Spent on Site`: consumer time on site in minutes
* `Age`: cutomer age in years
* `Area Income`: Avg. Income of geographical area of consumer
* `Daily Internet Usage`: Avg. minutes a day consumer is on the internet
* `Ad Topic Line`: Headline of the advertisement
* `City`: City of consumer
* `Male`: Whether or not consumer was male
* `Country`: Country of consumer
* `Timestamp`: Time at which consumer clicked on Ad or closed window
* `Clicked on Ad`: 0 or 1 indicated clicking on Ad

In [ ]:
data.info()

In [13]:
# Date format
data["Timestamp"] = pd.to_datetime(data["Timestamp"], format="%Y-%m-%d")

In [ ]:
data.info()

In [ ]:
data.describe()

### 2.2 Exploratory Data Analysis

In [ ]:
# Age repartition
plt.figure(figsize=(10, 8))
data.Age.hist(bins=data.Age.nunique())
plt.xlabel('Age')

In [ ]:
# Time on site
plt.figure(figsize=(10, 8))
data["Daily Time Spent on Site"].hist()
plt.xlabel("Daily Time Spent on Site")

In [ ]:
# Does younger people spend more time on site?
sns.jointplot(data["Daily Time Spent on Site"], data.Age)

In [ ]:
sns.jointplot(data["Daily Time Spent on Site"], data["Daily Internet Usage"])

In [ ]:
sns.pairplot(data, hue='Clicked on Ad')

In [ ]:
data2 = data[["Daily Time Spent on Site", "Daily Internet Usage", "Clicked on Ad"]]
sns.pairplot(data2, hue="Clicked on Ad")

### 2.3 Logistic Regression

Logistic regression is the classic linear classification algorithm for two-class problems.

#### 2.3.1 Theory

##### Logistic Regression

Logistic regression is named for the function used at the core of the method, the [logistic function](https://en.wikipedia.org/wiki/Logistic_function).

The logistic function, also called the **`Sigmoid function`** was developed by statisticians to describe properties of population growth in ecology, rising quickly and maxing out at the carrying capacity of the environment. It’s an S-shaped curve that can take any real-valued number and map it into a value between 0 and 1, but never exactly at those limits.

$$\frac{1}{1 + e^{-x}}$$

$e$ is the base of the natural logarithms and $x$ is value that you want to transform via the logistic function.

In [ ]:
x = np.linspace(-6, 6, num=1000)
plt.figure(figsize=(10, 6))
plt.plot(x, (1 / (1 + np.exp(-x))))
plt.title("Sigmoid Function")

The logistic regression equation has a very similar representation like linear regression. The difference is that the output value being modelled is binary in nature.

$$\hat{y}=\frac{e^{\beta_0+\beta_1x_1}}{1+\beta_0+\beta_1x_1}$$

or

$$\hat{y}=\frac{1.0}{1.0+e^{-\beta_0-\beta_1x_1}}$$

$\beta_0$ is the intecept term

$\beta_1$ is the coefficient for $x_1$

$\hat{y}$ is the predicted output with real value between 0 and 1. To convert this to binary output of 0 or 1, this would either need to be rounded to an integer value or a cutoff point be provided to specify the class segregation point.
***
##### Learning the Logistic Regression Model

The coefficients (Beta values b) of the logistic regression algorithm must be estimated from your training data. This is done using [maximum-likelihood estimation](https://en.wikipedia.org/wiki/Maximum_likelihood_estimation).

Maximum-likelihood estimation is a common learning algorithm used by a variety of machine learning algorithms, although it does make assumptions about the distribution of your data (more on this when we talk about preparing your data).

The best coefficients would result in a model that would predict a value very close to 1 (e.g. male) for the default class and a value very close to 0 (e.g. female) for the other class. The intuition for maximum-likelihood for logistic regression is that a search procedure seeks values for the coefficients (Beta values) that minimize the error in the probabilities predicted by the model to those in the data (e.g. probability of 1 if the data is the primary class).

We are not going to go into the math of maximum likelihood. It is enough to say that a minimization algorithm is used to optimize the best values for the coefficients for your training data. This is often implemented in practice using efficient numerical optimization algorithm (like the Quasi-newton method).

When you are learning logistic, you can implement it yourself from scratch using the much simpler gradient descent algorithm.

##### Prepare Data for Logistic Regression
The assumptions made by logistic regression about the distribution and relationships in your data are much the same as the assumptions made in linear regression.

Much study has gone into defining these assumptions and precise probabilistic and statistical language is used. My advice is to use these as guidelines or rules of thumb and experiment with different data preparation schemes.

Ultimately in predictive modeling machine learning projects you are laser focused on making accurate predictions rather than interpreting the results. As such, you can break some assumptions as long as the model is robust and performs well.

- **Binary Output Variable:** This might be obvious as we have already mentioned it, but logistic regression is intended for binary (two-class) classification problems. It will predict the probability of an instance belonging to the default class, which can be snapped into a 0 or 1 classification.
- **Remove Noise:** Logistic regression assumes no error in the output variable (y), consider removing outliers and possibly misclassified instances from your training data.
- **Gaussian Distribution:** Logistic regression is a linear algorithm (with a non-linear transform on output). It does assume a linear relationship between the input variables with the output. Data transforms of your input variables that better expose this linear relationship can result in a more accurate model. For example, you can use log, root, Box-Cox and other univariate transforms to better expose this relationship.
- **Remove Correlated Inputs:** Like linear regression, the model can overfit if you have multiple highly-correlated inputs. Consider calculating the pairwise correlations between all inputs and removing highly correlated inputs.
- **Fail to Converge:** It is possible for the expected likelihood estimation process that learns the coefficients to fail to converge. This can happen if there are many highly correlated inputs in your data or the data is very sparse (e.g. lots of zeros in your input data).

#### 2.3.2 Implementing Logistic Regression

In [ ]:
# Base rate
data[data["Clicked on Ad"] == 0]
#max(len(data[data["Clicked on Ad"] == 0])/len(data), len(data[data["Clicked on Ad"] == 1])/len(data))

In [ ]:
data.head()

In [ ]:
# Encode Country and City
data["Country"] = data.Country.astype('category').cat.codes
data["City"] = data.City.astype('category').cat.codes
data.head()

In [ ]:
# Select variables
X = data.drop(['Timestamp', 'Clicked on Ad', 'Ad Topic Line'], axis=1)
y = data['Clicked on Ad']
X

In [ ]:
y

In [ ]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train

In [ ]:
X_test

In [ ]:
# Fit Model
log_reg = LogisticRegressionCV(solver='lbfgs', cv=5, max_iter=1000)
log_reg.fit(X_train, y_train)
y_pred = log_reg.predict(X_test)
y_pred

In [ ]:
y_test.values

#### 2.3.3 Performance measurement

In [32]:
def evaluate(true, pred):
    precision = precision_score(true, pred)
    recall = recall_score(true, pred)
    f1 = f1_score(true, pred)
    print(f"CONFUSION MATRIX:\n{confusion_matrix(true, pred)}")
    print(f"ACCURACY SCORE:\n{accuracy_score(true, pred):.4f}")
    print(f"CLASSIFICATION REPORT:\n\tPrecision: {precision:.4f}\n\tRecall: {recall:.4f}\n\tF1_Score: {f1:.4f}")

In [ ]:
evaluate(y_test, y_pred)

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
print("True positives: " + str(tp))
print("True negatives: " + str(tn))
print("False positives: " + str(fp))
print("False negatives: " + str(fn))

###### Confusion matrix:
* true positives (93): people who clicked on ad and were classified as clicked on ad.
* true negatives (85): people who did not click on ad and were classified as did not click on ad.
* false positives (4): people who did not click on as and were classified as clicked on as.
* false negatives (18): people who clicked on ad and were classified as did not click on ad.

###### Accuracy score:
correct classifications / total = (85+93) / (85+93+18+4) = 0.89

###### Precision:
true positives / (true positives + false positives) = 93 / (93+4) = 0.9588

###### Recall:
true positives / (true positives + false negatives) = 93 / (93+18) = 0.8378

###### F1 score:
harmonic mean of precision and recall.


### 2.4 Logistic Regression with standardization


Standardization is helpful to give the same weight (or importance) to each predictor variable. The aim is to resize the s so that their mean equal 0 and their standard deviation equal 1.

In [ ]:
# Standardize features
standardize = StandardScaler()
standardize.fit(X_train, y_train)
# !!!IMPORTANT: we must fit on the training set, not on the whole set!!!

X_train_s = standardize.transform(X_train)
X_test_s = standardize.transform(X_test)

pd.DataFrame(X_train_s, columns=X_train.columns)#.describe()

In [ ]:
# Fit Logistic Regression and compute predictions
log_reg_s = LogisticRegressionCV(solver='lbfgs', cv=5, max_iter=1000)
log_reg_s.fit(X_train_s, y_train)
y_pred_s = log_reg_s.predict(X_test_s)

# Performance measurement
evaluate(y_test, y_pred_s)

## 3. Multi Class Regression
Let's now consider a classification problem with more than 2 target classes. For this we will use the iris data-set which has 3 target classes.

In [ ]:
iris = datasets.load_iris()
X = iris.data[:, :2] # we only take the first two features
y = iris.target
X[:10] # first 10 instances

In [ ]:
y # three classes

In [39]:
# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

In [ ]:
# Fit model
log_reg = LogisticRegressionCV(solver='lbfgs', cv=5, max_iter=1000)
log_reg.fit(X_train, y_train)

In [ ]:
# Accuracy of training set
log_reg.score(X_train, y_train)

In [ ]:
log_reg.score(X_test, y_test)
# accuracy_score(log_reg.predict(X_test), y_test)

In [ ]:
# True values
y_test

In [ ]:
# Predictions
log_reg.predict(X_test)

In [ ]:
log_reg.predict_proba(X_test)

In [ ]:
# Create DateFrame with probabilities, predictions, and true classes in test set
iris_LR_summary = pd.DataFrame(np.round(log_reg.predict_proba(X_test), 2), columns=["p(0)", "p(1)", "p(2)"])
iris_LR_summary["Prediction"] = log_reg.predict(X_test)
iris_LR_summary["True Class"] = y_test
iris_LR_summary

In [ ]:
# Confusion matrix - 4 errors out of 30 points
confusion_matrix(y_test, log_reg.predict(X_test))

In [ ]:
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = classes[unique_labels(y_true, y_pred)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

#     print(cm)

    fig, ax = plt.subplots(figsize=(10,7))
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')
    plt.ylim([-0.5, 2.5])

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    
    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout();
    return ax

# Plot non-normalized confusion matrix
plot_confusion_matrix(y_test, log_reg.predict(X_test), classes=iris.target_names,
                      title='Confusion matrix, without normalization')

In [ ]:
# Plot the decision boundaries for test set. For that, we will assign a 
# color to each point in the mesh [x_min, x_max]x[y_min, y_max].
x_min, x_max = X_test[:, 0].min() - .2, X_test[:, 0].max() + .2
y_min, y_max = X_test[:, 1].min() - .2, X_test[:, 1].max() + .2
h = .001  # step size in the mesh
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
Z = log_reg.predict(np.c_[xx.ravel(), yy.ravel()])

# Put the result into a color plot
Z = Z.reshape(xx.shape)
plt.figure(1, figsize=(15, 10))
plt.pcolormesh(xx, yy, Z, cmap=plt.cm.Paired)

# Plot also the training points
plt.scatter(X_test[:, 0], X_test[:, 1], c=y_test, edgecolors='k', cmap=plt.cm.Paired)
plt.xlabel('Sepal length', fontsize=15)
plt.ylabel('Sepal width', fontsize=15)

plt.xlim(xx.min(), xx.max())
plt.ylim(yy.min(), yy.max())
plt.xticks(())
plt.yticks(())

plt.show()

In [ ]:
# confusion matrix - training set

# Plot non-normalized confusion matrix
plot_confusion_matrix(y_train, log_reg.predict(X_train), classes=iris.target_names,
                      title='Confusion matrix, without normalization')

In [ ]:
# Plot the decision boundaries for training set. For that, we will assign a 
# color to each point in the mesh [x_min, x_max]x[y_min, y_max].
x_min, x_max = X_train[:, 0].min() - .2, X_train[:, 0].max() + .2
y_min, y_max = X_train[:, 1].min() - .2, X_train[:, 1].max() + .2
h = .01  # step size in the mesh
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
Z = log_reg.predict(np.c_[xx.ravel(), yy.ravel()])

# Put the result into a color plot
Z = Z.reshape(xx.shape)
plt.figure(1, figsize=(15, 10))
plt.pcolormesh(xx, yy, Z, cmap=plt.cm.Paired)

# Plot also the training points
plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, edgecolors='k', cmap=plt.cm.Paired)
plt.xlabel('Sepal length', fontsize=15)
plt.ylabel('Sepal width', fontsize=15)

plt.xlim(xx.min(), xx.max())
plt.ylim(yy.min(), yy.max())
plt.xticks(())
plt.yticks(())

plt.show()

## References:
* [Scikit Learn Library](https://scikit-learn.org/stable/supervised_learning.html#supervised-learning)
* [Logistic Regression for Machine Learning by Jason Brownlee PhD](https://machinelearningmastery.com/logistic-regression-for-machine-learning/)
* [Advertising - Logistic Regression](https://www.kaggle.com/arpitsomani/advertisement-logistic-regression#Predictions-and-Evaluations)